---
title: "Train anomaly detection model"
execute:
  echo: true
  enabled: false
  output: true
  warning: false
format:
    html:
        code-fold: false
---

In [1]:
#| echo: false
#| output: false
import os
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as pd
import numpy as np

import ray
from detect_common import *

ray.init(address='auto', ignore_reinit_error=True)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

2023-03-14 10:18:06,369	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 192.168.1.93:6379...
2023-03-14 10:18:06,377	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at 192.168.1.93:8265 


In [2]:
#| echo: false
#| output: false
p1version = "v5"
p2version = "v6"
p3version = "v4"

In [24]:
rw_2 = pd.read_csv(
    "/jupyter/warehouse/Jerabek2022Collection-unirec/unirec-csv-p100/"
    "unirec/DoH-Real-World/data/captured/pcap/"
    "HTTPS-20102021-10h.pcapng.trapcap.csv"
)


In [25]:
rw_2["time TIME_FIRST"] = rw_2["time TIME_FIRST"].apply(dateutil.parser.parse)

In [26]:
rw_2_min_ts = rw_2["time TIME_FIRST"].min()
rw_2_min_ts

Timestamp('2021-10-20 09:28:57.977011')

In [27]:
rw_2_end_ts = rw_2_min_ts + 10 * pd.Timedelta("2 minutes")

In [28]:
rw_2_m = rw_2[rw_2["time TIME_FIRST"] < rw_2_end_ts]

In [29]:
path_doh_ips = "/jupyter/kladovka/data/Jerabek2022Collection/doh_resolver_ip.csv"

cp = CacheableProcessing(100, 100, get_doh_ips(path_doh_ips))
rw_2_p = cp.process(rw_2_m, f"cached/rw-2m2.p3.{p3version}.saved")

In [30]:
rw_2_p["IsDoH"].value_counts()

False    3156
Name: IsDoH, dtype: int64

In [13]:
rw_1_p_min_ts = rw_1_p["time TIME_FIRST"].min()

In [18]:
f = '2min'

a1_t = rw_1_p.set_index(["time TIME_FIRST"])

# a1_t53 = a1_t[a1_t["uint16 DST_PORT"] == dst_port][["uint32 PACKETS", "uint32 PACKETS_REV"]]
# a1_t53["PACKETS"] = a1_t53["uint32 PACKETS"] + a1_t53["uint32 PACKETS_REV"]
df_resample = a1_t.groupby(pd.Grouper(freq=f, origin=rw_1_p_min_ts)).first()
times = df_resample.index.tolist()
ranges = list(zip(times, times[1:]))

df_calcs = []
for ran in ranges[:10]:
    print(ran)
    df_ran = a1_t[(a1_t.index > ran[0]) & (a1_t.index < ran[1])]
    df_calc = calc(df_ran, "IsDoH")
    df_calc["ts"] = ran[0]
    df_calcs.append(df_calc)

# data["df_calc"] = pd.concat(df_calcs).reset_index(drop=True)

(Timestamp('2021-10-04 10:18:57.444786', freq='2T'), Timestamp('2021-10-04 10:20:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:20:57.444786', freq='2T'), Timestamp('2021-10-04 10:22:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:22:57.444786', freq='2T'), Timestamp('2021-10-04 10:24:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:24:57.444786', freq='2T'), Timestamp('2021-10-04 10:26:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:26:57.444786', freq='2T'), Timestamp('2021-10-04 10:28:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:28:57.444786', freq='2T'), Timestamp('2021-10-04 10:30:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:30:57.444786', freq='2T'), Timestamp('2021-10-04 10:32:57.444786', freq='2T'))
(Timestamp('2021-10-04 10:32:57.444786', freq='2T'), Timestamp('2021-10-04 10:34:57.444786', freq='2T'))


In [23]:
df_ran

,ipaddr DST_IP,ipaddr SRC_IP,uint64 BYTES,uint64 BYTES_REV,uint64 LINK_BIT_FIELD,time TIME_LAST,macaddr DST_MAC,macaddr SRC_MAC,uint32 PACKETS,uint32 PACKETS_REV,uint16 DST_PORT,uint16 SRC_PORT,uint16 TLS_VERSION,uint8 DIR_BIT_FIELD,uint8 PROTOCOL,uint8 TCP_FLAGS,uint8 TCP_FLAGS_REV,int8* PPI_PKT_DIRECTIONS,uint8* PPI_PKT_FLAGS,string TLS_ALPN,bytes TLS_JA3,string TLS_SNI,uint16* PPI_PKT_LENGTHS,uint32* D_PHISTS_IPT,uint32* D_PHISTS_SIZES,uint32* S_PHISTS_IPT,uint32* S_PHISTS_SIZES,time* PPI_PKT_TIMES,uint16* PPI_PKT_LENGTHS_1,uint16* PPI_PKT_LENGTHS_-1,PPI_PKT_INTERVALS,PPI_PKT_INTERVALS_1,PPI_PKT_INTERVALS-1,IsDoH
time TIME_FIRST,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-04 10:32:57.533947,58.118.214.56,161.113.1.74,2010,4037,1,2021-10-04T10:32:57.582531,42:9c:6f:05:32:04,03:b8:2d:ba:cb:5a,11,15,443,59879,0,0,6,26,26,"[1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1...",[24|24|24|24|24|24|24|24|24|16|24|16|24|24],NaN,NaN,NaN,"[517, 212, 64, 92, 651, 207, 179, 80, 57, 350,...",[7|0|0|0|0|0|0|0],[0|0|1|2|2|1|0|2],[5|0|0|0|0|0|0|0],[0|1|0|2|1|0|2|0],[2021-10-04T10:32:57.566282|2021-10-04T10:32:5...,"[517, 64, 92, 651, 207, 31]","[212, 179, 80, 57, 350, 1111, 94, 1346]","[1.26, 5.058, 0.155, 0.131, 0.0, 0.481, 0.0710...","[1.26, 0.155, 0.131, 0.0, 0.481]","[5.058, 0.07100000000000001, 0.799, 0.212, 0.0...",False
2021-10-04 10:32:57.606320,209.251.64.226,30.25.78.1,52,52,1,2021-10-04T10:32:57.611590,a2:71:b3:29:89:22,1e:03:a1:2e:1d:f1,1,1,443,44980,0,0,6,16,16,[],[],NaN,NaN,NaN,[],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[],[],[],[],[],[],False
2021-10-04 10:32:57.911037,228.59.93.155,246.1.179.48,41,52,1,2021-10-04T10:32:57.916324,a2:71:b3:29:89:22,1e:03:a1:2e:1d:f1,1,1,443,61043,0,0,6,16,16,[1],[16],NaN,NaN,NaN,[1],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[1|0|0|0|0|0|0|0],[2021-10-04T10:32:57.911037],[1],[],[],[],[],False
2021-10-04 10:32:57.743516,84.113.70.202,15.142.52.163,52,52,1,2021-10-04T10:32:57.748781,a2:71:b3:29:89:22,1e:03:a1:2e:1d:f1,1,1,443,36512,0,0,6,16,16,[],[],NaN,NaN,NaN,[],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[],[],[],[],[],[],False
2021-10-04 10:32:57.884957,228.59.93.155,246.1.179.48,41,52,1,2021-10-04T10:32:57.890233,a2:71:b3:29:89:22,1e:03:a1:2e:1d:f1,1,1,443,54682,0,0,6,16,16,[1],[16],NaN,NaN,NaN,[1],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[0|0|0|0|0|0|0|0],[1|0|0|0|0|0|0|0],[2021-10-04T10:32:57.884957],[1],[],[],[],[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-04 10:33:59.013424,66.254.222.201,183.45.147.63,12630,84722,1,2021-10-04T10:36:54.321942,a2:71:b3:29:89:22,1e:03:a1:2e:1d:f1,309,335,443,61160,0,0,6,24,24,"[-1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1...",[24|24|24|24|24|24|24|24|24|24|24|24|24|24|24|...,NaN,NaN,NaN,"[125, 62, 54, 47, 101, 272, 94, 292, 172, 83, ...",[6|11|21|48|67|60|60|56],[0|0|32|117|90|63|25|3],[0|0|0|0|0|0|0|4],[0|0|5|0|0|0|0|0],[2021-10-04T10:33:59.193054|2021-10-04T10:33:5...,"[54, 54]","[125, 62, 47, 101, 272, 94, 292, 172, 83, 193,...","[736.328, 131.133, 113.645, 1455.074, 172.988,...","[113.645, 115.392]","[736.328, 131.133, 1455.074, 172.988, 652.169,...",False
2021-10-04 10:33:57.592477,253.17.88.220,226.239.36.48,190320,2096644,1,2021-10-04T10:36:53.678502,7a:4c:0b:75:a6:95,03:b8:2d:ba:cb:5a,444,1486,443,61650,0,0,6,24,24,"[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1...",[24|16|16|16|16|16|24|16|16|16|16|16|16|24|24|...,NaN,NaN,NaN,"[1438, 1460, 1460, 1460, 1460, 1460, 896, 1460...",[1351|0|0|0|0|0|2|117],[0|5|5|4|2|17|108|1330],[0|0|0|0|0|0|2|117],[0|0|0|0|0|0|0|120],[2021-10-04T10:33:57.592477|2021-10-04T10:33:5...,"[1438, 1438, 1438, 1438, 1438, 1438, 1438, 1438]","[1460, 1460, 1460, 1460, 1460, 896, 1460, 1460...","[106.469, 0.011, 0.009000000000000001, 0.011, ...","[106.469, 3.063, 3.383, 3.43, 3.169, 2.971, 3....","[0.011, 0.009000000000000001, 0.011, 0.01, 0.0...",False
2021

In [20]:
r = pd.concat(df_calcs).reset_index(drop=True)

In [22]:
r[r["dns_packets"] > 0]

,index,dns_packets,non_dns,rate,rate_log,ts


In [ ]:
import matplotlib.pyplot as plt

for i, data in models.items():
    model, normed, skips_and_weights = data["params"]
    name = (
        ('normed-' if normed else 'unnormed-') + 
        '-'.join(str(p) for p in skips_and_weights)
    )

    fig, axs = plt.subplots(ncols=1, figsize=(12, 4))
    print(data["params"])
    df_calcs = data["df_calc"]
    for host in local_ips:
        df_calc_host = df_calcs[df_calcs["index"] == host].set_index("ts")
        df_calc_host["rate_log"].plot(label=f'{host}', ax=axs)

    plt.title(f"{model} {name}")
    plt.legend()
    plt.show()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)